In [76]:
%tensorflow_version 1.x

In [77]:
# Importando as bibliotecas necessárias:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [78]:
n_epochs = 20
n_batches = 50
n_epochs = 40
batch_size = 50

In [79]:
n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
learning_rate = 0.01

In [80]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [81]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [82]:
reset_graph()

In [83]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [84]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)

In [85]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)

In [86]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [87]:
with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)

In [88]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [89]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [90]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 0.96 Validation accuracy: 0.9564
1 Batch accuracy: 0.94 Validation accuracy: 0.9714
2 Batch accuracy: 0.98 Validation accuracy: 0.972
3 Batch accuracy: 0.96 Validation accuracy: 0.9762
4 Batch accuracy: 0.98 Validation accuracy: 0.9782
5 Batch accuracy: 1.0 Validation accuracy: 0.9788
6 Batch accuracy: 1.0 Validation accuracy: 0.9808
7 Batch accuracy: 1.0 Validation accuracy: 0.9806
8 Batch accuracy: 1.0 Validation accuracy: 0.9812
9 Batch accuracy: 0.98 Validation accuracy: 0.9816
10 Batch accuracy: 1.0 Validation accuracy: 0.9838
11 Batch accuracy: 1.0 Validation accuracy: 0.9842
12 Batch accuracy: 1.0 Validation accuracy: 0.9826
13 Batch accuracy: 1.0 Validation accuracy: 0.9826
14 Batch accuracy: 1.0 Validation accuracy: 0.9838
15 Batch accuracy: 1.0 Validation accuracy: 0.9848
16 Batch accuracy: 1.0 Validation accuracy: 0.9842
17 Batch accuracy: 1.0 Validation accuracy: 0.9838
18 Batch accuracy: 1.0 Validation accuracy: 0.9842
19 Batch accuracy: 1.0 Validation ac

In [91]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [92]:
m, n = X_train.shape
logdir = log_dir("mnist_dnn")
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [93]:
n_epochs = 10001
batch_size = 50
n_batches = int(np.ceil(m / batch_size))

checkpoint_path = "/tmp/my_deep_mnist_model.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./my_deep_mnist_model"

best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 50

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # if the checkpoint file exists, restore the model and load the epoch number
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)

    for epoch in range(start_epoch, n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: X_valid, y: y_valid})
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        if epoch % 5 == 0:
            print("Epoch:", epoch,
                  "\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),
                  "\tLoss: {:.5f}".format(loss_val))
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
            if loss_val < best_loss:
                saver.save(sess, final_model_path)
                best_loss = loss_val
            else:
                epochs_without_progress += 5
                if epochs_without_progress > max_epochs_without_progress:
                    print("Early stopping")
                    break

Training was interrupted. Continuing at epoch 66
INFO:tensorflow:Restoring parameters from /tmp/my_deep_mnist_model.ckpt
Epoch: 70 	Validation accuracy: 98.420% 	Loss: 0.08523
Epoch: 75 	Validation accuracy: 98.380% 	Loss: 0.08594
Epoch: 80 	Validation accuracy: 98.400% 	Loss: 0.08652
Epoch: 85 	Validation accuracy: 98.380% 	Loss: 0.08717
Epoch: 90 	Validation accuracy: 98.400% 	Loss: 0.08760
Epoch: 95 	Validation accuracy: 98.380% 	Loss: 0.08834
Epoch: 100 	Validation accuracy: 98.400% 	Loss: 0.08874
Epoch: 105 	Validation accuracy: 98.400% 	Loss: 0.08947
Epoch: 110 	Validation accuracy: 98.360% 	Loss: 0.09004
Epoch: 115 	Validation accuracy: 98.400% 	Loss: 0.09028
Epoch: 120 	Validation accuracy: 98.420% 	Loss: 0.09078
Epoch: 125 	Validation accuracy: 98.380% 	Loss: 0.09101
Early stopping
